<h1 align="center"> ISYE 6501- Introduction to Analytics Modeling
<h2 align="center"> Week 10 HW - Missing Data and Optimization (Question 15.2)
<h3 align="center"> by: Martin Cisneros

# Contents  
   
[Introduction](#intro)  

[Simple optimization model: Part One](#model1)

[Full optimization model with added constraints: Part Two](#model2)

[TLDR; Results](#results) 


<a id='intro'></a>
# Introduction

In the videos, we saw the “diet problem”. (The diet problem is one of the first large-scale optimization problems to be studied in practice. Back in the 1930’s and 40’s, the Army wanted to meet the nutritional requirements of its soldiers while minimizing the cost.) In this homework you get to solve a diet problem with real data. The data is given in the file diet.xls.

- Part One: Formulate an optimization model (a linear program) to find the cheapest diet that satisfies the maximum and minimum daily nutrition constraints, and solve it using PuLP. Turn in your code and the solution. (The optimal solution should be a diet of air-popped popcorn, poached eggs, oranges, raw iceberg lettuce, raw celery, and frozen broccoli. UGH!)
- Part Two: Please add to your model the following constraints (which might require adding more variables) and solve the new model:
  
        a. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)
  
        b. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

        c. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. {If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!}

If you want to see what a more full-sized problem would look like, try solving your models for the file dietlarge.xls, which is a low-cholesterol diet model (rather than minimizing cost, the goal is to minimize cholesterol intake). I don’t know anyone who’d want to eat this diet, the optimal solution includes dried chrysanthemum garland, raw beluga whale flipper, freeze-dried parsley, etc. which shows why it’s necessary to add additional constraints beyond the basic ones we saw in the video!


<a id='model1'></a>
# Simple optimization model: Part One

<a id='libs1'></a>
### Import Libraries

In [243]:
from pulp import *
import pandas as pd

###  Retrieve Data

In [244]:
data = pd.read_excel("diet.xls", header = 0) 

data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [245]:
# Create data table
dataTable = data[0:64] 
dataTable.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [246]:
# Create list to pull column names
dataTable = dataTable.values.tolist() 
columnNames = list(data.columns.values) 
columnNames

['Foods',
 'Price/ Serving',
 'Serving Size',
 'Calories',
 'Cholesterol mg',
 'Total_Fat g',
 'Sodium mg',
 'Carbohydrates g',
 'Dietary_Fiber g',
 'Protein g',
 'Vit_A IU',
 'Vit_C IU',
 'Calcium mg',
 'Iron mg']

Nutrients are columns 3 to 13

In [247]:
# min and max nutrient values
minvalue = data[65:66].values.tolist() 
maxvalue = data[66:67].values.tolist() 

### Create relevant vectors

In [248]:
foods = [j[0] for j in dataTable] 
cost = dict([(j[0], float(j[1])) for j in dataTable]) 
nutrients = []
for i in range(0,11): 
    nutrients.append(dict([(j[0], float(j[i+3])) for j in dataTable])) # only pull nutrients

### Create LP Problem

This problem is a minimization problem to find the lowest cost; ie. need LpMinimize.

In [249]:
prob = LpProblem('Food optimization', LpMinimize)

### Define variables

Lower limit of each variable is 0, since we can't eat negative amounts of anything.

In [250]:
foodVars = LpVariable.dicts("Foods", foods, 0)

### Create objective function

Note that the first function we add is taken to be the objective function

In [251]:
prob += lpSum([cost[f] * foodVars[f] for f in foods]), 'Total Cost'

### Add constraints for each nutrient

In [252]:
for i in range(0,11): # for loop running through each nutrient: 11 times starting with 0
    prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) >= minvalue[0][i+3], 'min nutrient ' + columnNames[i]
    prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) <= maxvalue[0][i+3], 'max nutrient ' + columnNames[i]

### Solve the simple optimization problem

In [253]:
prob.solve()

print()
print("Solution:")
print()
for var in prob.variables():
    if var.varValue > 0:
        print(str(var.varValue)+" units of "+str(var).replace('Foods_','') )
print()
print("Cost of food = $%.2f" % value(prob.objective)) 


Solution:

52.64371 units of Celery,_Raw
0.25960653 units of Frozen_Broccoli
63.988506 units of Lettuce,Iceberg,Raw
2.2929389 units of Oranges
0.14184397 units of Poached_Eggs
13.869322 units of Popcorn,Air_Popped

Cost of food = $4.34


<a id='model2'></a>
# Full optimization model with added constraints: Part Two

### Add additional variables

1. Create binary integer variables for whether a food is eaten
2. We can't eat negative amounts of anything

In [254]:
foodie = LpVariable.dicts("foods", foods, 0)
foodie_bool = LpVariable.dicts("foodies",foods,0,1,LpBinary) 

### Create objective function

I am okay with re-writing the previous objective since I've recorded Part one results. I am re-using the previously set 'prob'

In [255]:
prob += lpSum([cost[f] * foodie[f] for f in foods]), 'Total Cost'

/Users/mcisneros/anaconda3/lib/python3.6/site-packages/pulp/pulp.py:1348: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


### Add constraints for each nutrient

In [256]:
for i in range(0,11): 
    prob += lpSum([nutrients[i][j] * foodie[j] for j in foods]) >= minvalue[0][i+3], 'min nutrient 2' + columnNames[i]
    prob += lpSum([nutrients[i][j] * foodie[j] for j in foods]) <= maxvalue[0][i+3], 'max nutrient 2' + columnNames[i]
    

### Add additional constraints

1. If a food is selected, then a minimum of 1/10 serving must be chosen. (Hint: now you will need two variables for each food i: whether it is chosen, and how much is part of the diet. You’ll also need to write a constraint to link them.)

2. Many people dislike celery and frozen broccoli. So at most one, but not both, can be selected.

3. To get day-to-day variety in protein, at least 3 kinds of meat/poultry/fish/eggs must be selected. {If something is ambiguous (e.g., should bean-and-bacon soup be considered meat?), just call it whatever you think is appropriate – I want you to learn how to write this type of constraint, but I don’t really care whether we agree on how to classify foods!}

- 'Bologna,Turkey': Foods_Bologna,Turkey,
- 'Chicknoodl Soup': Foods_Chicknoodl_Soup,
- 'Frankfurter, Beef': Foods_Frankfurter,_Beef,
- 'Ham,Sliced,Extralean': Foods_Ham,Sliced,Extralean,
- 'Hamburger W/Toppings': Foods_Hamburger_W_Toppings,
- 'Hotdog, Plain': Foods_Hotdog,_Plain,
- 'Pizza W/Pepperoni': Foods_Pizza_W_Pepperoni,
- 'Pork': Foods_Pork,
- 'Roasted Chicken': Foods_Roasted_Chicken,
- 'Sardines in Oil': Foods_Sardines_in_Oil,
- 'Taco': Foods_Taco,
- 'Tofu': Foods_Tofu,
- 'White Tuna in Water': Foods_White_Tuna_in_Water}

In [257]:
# If a food is eaten, must eat at least 0.1 units

for food in foods:
    prob += foodie[food] >= 0.1 * foodie_bool[food]

# If any of a food is eaten, must ensure it's binary variable must be 1

for food in foods:
    prob += foodie_bool[food] >= foodie[food]*0.0000001 

# At most one, but not both, can be selected (celery or frozen broccoli)

prob += foodie_bool['Frozen Broccoli'] + foodie_bool['Celery, Raw'] <= 1 

# at least 3 kinds of meat/poultry/fish/eggs (see above)

prob += foodie_bool['Bologna,Turkey'] + foodie_bool['Chicknoodl Soup'] \
        + foodie_bool['Frankfurter, Beef'] + foodie_bool['Ham,Sliced,Extralean'] \
        + foodie_bool['Hamburger W/Toppings'] + foodie_bool['Hotdog, Plain'] \
        + foodie_bool['Pizza W/Pepperoni'] + foodie_bool['Pork'] \
        + foodie_bool['Roasted Chicken'] \
        + foodie_bool['Sardines in Oil'] + foodie_bool['Taco'] \
        + foodie_bool['Tofu'] + foodie_bool['White Tuna in Water'] >= 3


### Solve the full optimization problem

In [258]:
prob.solve()

print()
print("Solution:")
for var in prob.variables():
    if var.varValue > 0 and "foodies" not in var.name: # Print non binary variables
        print(str(var.varValue)+" units of "+str(var).replace('foods','') )
print()
print("Cost of food = $%.2f" % value(prob.objective)) 


Solution:
13.243507 units of Foods_Apple,Raw,W_Skin
51.801956 units of Foods_Celery,_Raw
1.0694444 units of Foods_Kiwifruit,Raw,Fresh
88.139375 units of Foods_Lettuce,Iceberg,Raw
0.13024476 units of Foods_Poached_Eggs
1.9710008 units of Foods_Popcorn,Air_Popped
0.01888555 units of Foods_Roasted_Chicken
1.4571319 units of Foods_Tofu
0.1 units of _Bologna,Turkey
41.86398 units of _Celery,_Raw
0.1 units of _Frankfurter,_Beef
83.547931 units of _Lettuce,Iceberg,Raw
3.0825324 units of _Oranges
1.8656401 units of _Peanut_Butter
0.11560284 units of _Poached_Eggs
13.2598 units of _Popcorn,Air_Popped
0.1 units of _Tofu

Cost of food = $4.55


<a id='results'></a>
# TLDR; Results

Simple optimization model: Part One 

- 52.64371 units of raw celery
- 0.25960653 units of frozen broccoli
- 63.988506 units of raw iceberg lettuce
- 2.2929389 units of oranges
- 0.14184397 units of poached eggs
- 13.869322 units of air-popped popcorn

Cost of food = _$4.34_

Full optimization model with added constraints: Part Two

- 13.243507 units of Foods_Apple,Raw,W_Skin
- 51.801956 units of Foods_Celery,_Raw
- 1.0694444 units of Foods_Kiwifruit,Raw,Fresh
- 88.139375 units of Foods_Lettuce,Iceberg,Raw
- 0.13024476 units of Foods_Poached_Eggs
- 1.9710008 units of Foods_Popcorn,Air_Popped
- 0.01888555 units of Foods_Roasted_Chicken
- 1.4571319 units of Foods_Tofu
- 0.1 units of _Bologna,Turkey
- 41.86398 units of _Celery,_Raw
- 0.1 units of _Frankfurter,_Beef
- 83.547931 units of _Lettuce,Iceberg,Raw
- 3.0825324 units of _Oranges
- 1.8656401 units of _Peanut_Butter
- 0.11560284 units of _Poached_Eggs
- 13.2598 units of _Popcorn,Air_Popped
- 0.1 units of _Tofu

Cost of food = _$4.55_


